## Index of the Process performed

## 1. Load Data
## 2. load Model
## 3. Before Training
## 4. After Training
## 5. Upload Model in Hugging face

In [1]:
## unsloth from git

!pip install huggingface_hub ipython "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git" "unsloth[conda] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-mbo61ecr/unsloth_e237fd426a32456eba7f9d2c30ba2db1
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-mbo61ecr/unsloth_e237fd426a32456eba7f9d2c30ba2db1
  Resolved https://github.com/unslothai/unsloth.git to commit ec19e61c854dcf9104386fa63fc6c4f2944d4f35
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
## transformers
!pip install "git+https://github.com/huggingface/transformers.git"


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-8nj9e589
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-8nj9e589
  Resolved https://github.com/huggingface/transformers.git to commit 8c12690cecbb97e187861e386f7a0ac790e4236c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 45.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.41.0.dev0-py3-none-any.whl size=9012708 sha256=1836844ffb4de612efc8bb1d7822230ea03f6e1dbfa702f8af8d01f798650d7a
  Stored in directory: /tmp/pip-ephem-wheel-cache-2xfacbpg/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      

In [3]:
## trl
!pip install trl


In [4]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install for  Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [42]:
# 1. Importing and configurations
import os
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

In [43]:
max_seq_length = 2048

Make a choice of Datasets to fine tune the existing model

In [44]:
## Dataset 1 OIG
#url = "https://huggingface.co/datasets/laion/OIG/resolve/main/unified_chip2.jsonl"
#dataset = load_dataset("json", data_files = {"train" : url}, split = "train")

In [45]:
## Dataset 2 IMDB
dataset = load_dataset("imdb", split="train")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [47]:
# 2. Load Mistral or LLAMA model or suitable Model Architecture

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",
    #model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)


==((====))==  Unsloth: Fast Mistral patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


ValueError: 
                    Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the
                    quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules
                    in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to
                    `from_pretrained`. Check
                    https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                    for more details.
                    

In [ ]:
## peft for model

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length
)

In [ ]:
## train model on the custom dataset
## model fine tuning

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "unsloth-test",
        optim = "adamw_8bit",
        seed = 3407,
    ),
)
trainer.train()

In [ ]:
## Generate inference based on the input promt / test to the fine tuned model

def generate_text_big(text, max_new_tokens_1):
    inputs = tokenizer(text, return_tensors="pt").to("cuda:0")
    #outputs = model.generate(**inputs, max_new_tokens=20)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens_1, use_cache=True)
    outputs_1 = tokenizer.batch_decode(outputs)
    #print(tokenizer.decode(outputs[0], skip_special_tokens=True))

    return outputs_1

In [ ]:
model.save_pretrained("lora_model")

In [ ]:
dataset[0]

In [ ]:
dataset[1]

In [ ]:
dataset[2]

In [ ]:
dataset[200]

In [ ]:
dataset[150]

In [ ]:
outputs_1 = generate_text_big("Is it okay to drink a lot of soda?", 50)

outputs_1

In [ ]:

outputs_2 = generate_text_big("gaining an understanding of the banking industry", 100)

outputs_2

In [ ]:
outputs_3 = generate_text_big("Make a detailed written description of a spooky scene outside town with an ornate gate and headstones", 50)

outputs_3

In [ ]:
## Original model without finetuning on Dataset

FastLanguageModel.for_inference(model)

In [ ]:
# 3 Before training

def generate_text_big(text):
    inputs = tokenizer(text, return_tensors="pt").to("cuda:0")
    #outputs = model.generate(**inputs, max_new_tokens=20)
    outputs = model.generate(**inputs, max_new_tokens=200)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print("Before training\n")